In [1]:
###############################################################################
# Common parameters
###############################################################################
'''
year_plan, month_plan, l_holiday  = 2024, 6, []
year_plan, month_plan, l_holiday  = 2024, 7, [15]
year_plan, month_plan, l_holiday  = 2024, 8, [12]
year_plan, month_plan, l_holiday  = 2024, 9, [16, 23]
year_plan, month_plan, l_holiday  = 2024, 10, [14]
year_plan, month_plan, l_holiday  = 2024, 11, [4]
year_plan, month_plan, l_holiday  = 2024, 12, [28, 29, 30, 31]
'''
year_plan, month_plan, l_holiday  = 2025, 2, [11, 24]
l_date_ect_cancel = []

from script.parameter import *

In [9]:
###############################################################################
# Create Google form
###############################################################################
from script.form import *
d_cal, d_date_duty, s_cnt_duty, s_cnt_class_duty, d_cal_duty, d_form =\
    prepare_form(lp_root, year_plan, month_plan, l_holiday, l_date_ect_cancel, l_day_ect, day_em, l_week_em, l_class_duty, dict_duty, dict_score_duty, dict_duty_jpn, dict_title_duty, dict_class_duty)

In [5]:
###############################################################################
# Collect google form response
###############################################################################
from script.collect import *
str_member_missing, str_mail_missing, d_availability, d_info, d_member =\
    collect_availability(lp_root, year_plan, month_plan, id_sheet_response, dict_jpnday, dict_duty_jpn)

Missing members and emails:
大熊 彩子, 朝重 菜々美
okuma.ayako@gmail.com, mochikura82@gmail.com
Requests:
市橋 香代 いつもありがとうございます。


In [ ]:
###############################################################################
# Optimize assignment count and assign members
###############################################################################
from script.assign import *
# Assignment count optimization
dict_c_diff_score_current = {'ampm': 0.001, 'daynight': 0.001, 'ampmdaynight': 0.001, 'oc': 0.001, 'ect': 0.01}
dict_c_diff_score_total = {'ampm': 0.01, 'daynight': 0.01, 'ampmdaynight': 0.01, 'oc': 0.01, 'ect': 0.1}
#dict_c_diff_score_total = {'ampm': 0.01, 'daynight': 0.01, 'ampmdaynight': 1.0, 'oc': 0.01, 'ect': 0.1}
# Individual assignment
# Parameters for avoiding/penalizing close duties
#dict_closeduty = {'daynight': {'l_duty': ['day', 'ocday', 'night', 'emnight', 'ocnight'], 'thr_hard': 2, 'thr_soft': 5}, # 1: avoid within same day, 2: avoid within 2 continuous days
#                  'ect':      {'l_duty': ['ect'],                                         'thr_hard': 1, 'thr_soft': 4},
#                  'ampm':     {'l_duty': ['am', 'pm'],                                    'thr_hard': 1, 'thr_soft': 2}}
dict_closeduty = {'daynight': {'l_duty': ['day', 'ocday', 'night', 'emnight', 'ocnight'], 'thr_hard': 0, 'thr_soft': 5}, # 1: avoid within same day, 2: avoid within 2 continuous days
                  'ect':      {'l_duty': ['ect'],                                         'thr_hard': 1, 'thr_soft': 4},
                  'ampm':     {'l_duty': ['am', 'pm'],                                    'thr_hard': 1, 'thr_soft': 2}}
#c_assign_suboptimal, c_cnt_deviation, c_closeduty = 0.001, 0.001, 0.1
c_assign_suboptimal, c_cnt_deviation, c_closeduty = 0.00001, 0.1, 0.00001
#c_assign_suboptimal, c_cnt_deviation, c_closeduty = 0.0001, 0.1, 0.01
l_date_duty_fulltime = []
# Count deviation penalization mode. student twice-assignation is limited elsewhere
type_limit = 'soft'  # 'hard': never exceed, 'soft': outlier penalized, 'ignore': no penalty
l_date_duty_skip_manual = ['5_ect', '12_ect', '26_ect']
'''
#l_date_duty_skip_manual = ['23_'] # All duties starting with 23_
#l_date_duty_skip_manual = ['23_am']
'''
d_assign, d_assign_date_print, d_assign_member, d_deviation, d_score_print, d_closeduty =\
    optimize_count_and_assign(lp_root, year_plan, month_plan, year_start, month_start, l_class_duty, dict_c_diff_score_current,
                              dict_c_diff_score_total, l_date_duty_skip_manual, dict_closeduty, ll_avoid_adjacent, l_title_fulltime,
                              l_date_duty_fulltime, type_limit, c_assign_suboptimal, c_cnt_deviation, c_closeduty, dict_score_duty, dict_score_class)

Assignment count optimization
------------------------------------------------------------
Done, losses: 12.19(non-OC), 0.0(OC)
Member assignment optimization
------------------------------------------------------------
No member available for: ['1_day', '4_pm', '7_pm', '8_ocday', '8_ocnight', '19_pm', '23_night']
of which ['1_day', '4_pm', '7_pm', '19_pm', '23_night'] are not OC
Manually assigned member(s) for: ['1_day', '23_day', '15_night', '24_night']
1_day  manually set to  34.0
23_day  manually set to  42.0
15_night  manually set to  10.0
24_night  manually set to  7.0
In total, skipping assignment for: ['4_pm', '7_pm', '19_pm', '23_night']
------------------------------------------------------------
[ERROR] Failed to solve. Entering troubleshooting mode to determin which duty caused the failure。
[TROUBLESHOOTING] iteration 1 test skipping 95 duties.
[TROUBLESHOOTING] iteration 2 test skipping 76 duties.
[TROUBLESHOOTING] iteration 3 test skipping 60 duties.
[TROUBLESHOOTING] ite

In [ ]:
###############################################################################
# Notify Google calendar
###############################################################################
from script.notify import *
update_calendar(lp_root, year_plan, month_plan, l_scope, dict_time_duty, t_sleep)

In [ ]:
###############################################################################
# Collect replacement application
###############################################################################
from script.replace import *
d_replace_checked = check_replacement(lp_root, year_plan, month_plan, sheet_id)

In [ ]:
###############################################################################
# Apply checked replacement plan
###############################################################################
d_assign, d_assign_date_print, d_assign_member, d_deviation, d_deviation_summary, d_score_current, d_score_total, d_score_print =\
    replace_assignment(lp_root, year_plan, month_plan, dict_score_duty, l_class_duty, d_replace_checked)